In [1]:
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from std_srvs.srv import *
import sys
import select
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, PlanningActionResult
from actionlib_msgs.msg import GoalStatus
from sensor_msgs.msg import LaserScan
import math
import ipywidgets as widgets
from IPython.display import display 
from ipywidgets import Layout, HBox
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, interactive, fixed, interact_manual
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
import matplotlib as mpl
from assignment_2_2023.srv import Last_target
import re

In [2]:
# Define state descriptions
state_desc_ = {
    0: 'Go to point',
    1: 'Wall following',
    2: 'Done',
    6: 'Goal cancelled'
}

In [3]:
# Check if ROS node has already been initialized
if not rospy.core.is_initialized():
    rospy.init_node('set_target_client')

In [4]:
# Declare position_display in the global scope
position_display = widgets.HTML(value="<b><u>Position:</u></b> <b>X</b>: 0, <b>Y</b>: 0<br><b><u>Velocity:</u></b> <b>Linear</b>: 0, <b>Angular</b>: 0")

status_text = widgets.Label(value="Current Status: Unknown")

In [5]:
# Publisher node function to handle position values and publish them
def publisher_node(msg):
    global pub, old_my_pos_and_vel
    
    # Get the actual position and velocity
    actual_pos = msg.pose.pose.position
    actual_vel_linear = msg.twist.twist.linear
    actual_vel_angular = msg.twist.twist.angular
    
    # Create a Vel message
    my_pos_and_vel = Vel()
    my_pos_and_vel.pos_x = actual_pos.x
    my_pos_and_vel.pos_y = actual_pos.y
    my_pos_and_vel.vel_x = actual_vel_linear.x
    my_pos_and_vel.vel_z = actual_vel_angular.z


    if (abs(my_pos_and_vel.pos_x - old_my_pos_and_vel.pos_x) > 0.05):
        old_my_pos_and_vel.pos_x = my_pos_and_vel.pos_x
    
    
    if (abs(my_pos_and_vel.pos_y -  old_my_pos_and_vel.pos_y) > 0.05):
        old_my_pos_and_vel.pos_y = my_pos_and_vel.pos_y

    if (abs(my_pos_and_vel.vel_x - old_my_pos_and_vel.vel_x) > 0.05):
        old_my_pos_and_vel.vel_x = my_pos_and_vel.vel_x

    if (abs(my_pos_and_vel.vel_z - old_my_pos_and_vel.vel_z) > 0.05):
        old_my_pos_and_vel.vel_z = my_pos_and_vel.vel_z
    
    # Publish the position and velocity
    pub.publish(my_pos_and_vel)

    new_pos_x = "{:.3f}".format(old_my_pos_and_vel.pos_x)
    new_pos_y = "{:.3f}".format(old_my_pos_and_vel.pos_y)
    new_vel_x = "{:.3f}".format(old_my_pos_and_vel.vel_x)
    new_vel_z = "{:.3f}".format(old_my_pos_and_vel.vel_z)    
    
    # Update position and velocity display
    position_display.value = f"<b>Position:</b> X: {new_pos_x}, Y: {new_pos_y}<br><b>Velocity:</b> Linear: {new_vel_x}, Angular: {new_vel_z}"
    

In [6]:
global old_my_pos_and_vel
old_my_pos_and_vel = Vel()

old_my_pos_and_vel.pos_x = 0.0
old_my_pos_and_vel.pos_y = 0.0
old_my_pos_and_vel.vel_x = 0.0
old_my_pos_and_vel.vel_z = 0.0

# Initialize publisher
pub = rospy.Publisher('/pos_vel', Vel, queue_size=10)

# Initialize subscriber
sub_from_Odom = rospy.Subscriber("/odom", Odometry, publisher_node)

In [7]:
def set_goal(input_x, input_y):
    goal = assignment_2_2023.msg.PlanningGoal()
    
    # Update ros parameters
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)

    # Set goal parameters
    goal.target_pose.pose.position.x = rospy.get_param('/des_pos_x')
    goal.target_pose.pose.position.y = rospy.get_param('/des_pos_y')
    #goal.target_pose.pose.position.x = input_x
    #goal.target_pose.pose.position.y = input_y

    # send goal to the service
    client.send_goal(goal)
    rospy.loginfo("Inserted goal: des_x = %f, des_y = %f", input_x, input_y)

In [8]:
def cancel_goal():
    if client.get_state() == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        rospy.loginfo("Goal cancelled")
    else:
        rospy.loginfo("No active goal to cancel")

In [9]:
# Update status_update function
def status_update(msg):
    global current_status
    if msg.status_list:
        current_status = msg.status_list[0].status
        # print("Received current status:", current_status)
        if current_status in state_desc_:
            status_text.value = f"Current Status: {state_desc_[current_status]} ({current_status})"
        else:
            status_text.value = f"Current Status: Unknown ({current_status})"
    else:
        status_text.value = "Current Status: Unknown"

In [10]:
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

True

In [11]:
# Create widgets for setting goal
goal_x = widgets.FloatSlider(min=-15.0, max=15.0, step=0.1, description='desired x:')
goal_y = widgets.FloatSlider(min=-15.0, max=15.0, step=0.1, description='desired y:')
set_goal_button = widgets.Button(description="Set Goal")
cancel_goal_button = widgets.Button(description="Cancel Goal")

[INFO] [1712669761.768655, 1524.028000]: Inserted goal: des_x = 4.800000, des_y = -2.700000
[INFO] [1712669800.652297, 1546.226000]: Goal cancelled


In [ ]:
# Create widgets for status update
# status_text = widgets.Label(value="Current Status: Unknown")
# Create widget for displaying position and velocity
# position_display = widgets.HTML(value="<b>Position:</b> X: 0, Y: 0<br><b>Velocity:</b> Linear: 0, Angular: 0")

In [12]:
# Define actions for button clicks
def set_goal_click(b):
    set_goal(goal_x.value, goal_y.value)

def cancel_goal_click(b):
    cancel_goal()

In [13]:
# Assign actions to buttons
set_goal_button.on_click(set_goal_click)
cancel_goal_button.on_click(cancel_goal_click)

In [14]:
# Subscribe to goal status updates
goal_status_sub = rospy.Subscriber('/reaching_goal/status', GoalStatus, status_update)

In [17]:
# Style widgets 
goal_x.layout.width = '400px'
goal_y.layout.width = '400px'
set_goal_button.style.button_color = 'lightblue'
cancel_goal_button.style.button_color = 'lightcoral'
status_text.layout.margin = '20px 0'
position_display.layout.margin = '20px 0'

layout = widgets.Layout(display='flex', flex_flow='column', align_items='center')

# Display widgets
display(widgets.VBox([goal_x, goal_y, set_goal_button, cancel_goal_button, status_text, position_display], layout=layout))